In [1]:
!pip install --upgrade pip
!pip install pandas==1.3.1
!pip install nltk
import nltk
nltk.download('stopwords')
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numba
import os
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import pandas as pd
import urllib.request
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import json
from IPython.display import Javascript
# json 파일 업로드 및 읽기
# file_path_list = ["/Users/choeseoyeong/Downloads/kickstarter-onlydata-2015-1-1-2016-1-1.json","/Users/choeseoyeong/Downloads/kickstarter-onlydata-2016-1-1-2017-1-1.json","/Users/choeseoyeong/Downloads/kickstarter-onlydata-2017-1-1-2018-1-1.json","/Users/choeseoyeong/Downloads/kickstarter-onlydata-2018-1-1-2019-1-1.json","/Users/choeseoyeong/Downloads/kickstarter-onlydata-2019-1-1-2020-1-1.json","/Users/choeseoyeong/Downloads/kickstarter-onlydata-2020-1-1-2021-1-1.json","/Users/choeseoyeong/Downloads/kickstarter-onlydata-2021-1-1-2022-1-1.json","/Users/choeseoyeong/Downloads/kickstarter-onlydata-2022-1-1-2023-1-1.json","/Users/choeseoyeong/Downloads/kickstarter-onlydata-2023-1-1-2024-1-1.json","/Users/choeseoyeong/Downloads/kickstarter_filtered_total.json" ]
def process_json_files(input_folder_path, output_folder_path):
    # 입력 폴더 내의 모든 JSON 파일을 처리
    for file_name in os.listdir(input_folder_path):
        if file_name.endswith('.json'):
            input_file_path = os.path.join(input_folder_path, file_name)
            output_file_path = os.path.join(output_folder_path, file_name.replace('.json', '_ESG_scores.json'))
            process_json_file(input_file_path, output_file_path)

def process_json_file(input_file_path, output_file_path):
    data_array = []
    with open(input_file_path, 'r') as file:
            try:
                data_array = json.load(file)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                print(input_file_path)
    # 데이터를 저장할 리스트
    dataset = []

    # 데이터에서 ['name', 'blurb', 'slug', 'creator name'] 형태의 데이터 리스트 추출
    for data in data_array:
        datatotal = []
        if 'name' in data:
            datatotal.append(data['name'].replace("'", ""))
        if 'blurb' in data:
            datatotal.append(data['blurb'].replace("'", ""))
        if 'slug' in data:
            datatotal.append(data['slug'].replace("'", ""))
        if 'creator' in data and 'name' in data['creator']:
            datatotal.append(data['creator']['name'].replace("'", ""))
        dataset.append(datatotal)
    list1 = ['air', 'biofertilizer', 'biogas', 'carbon', 'climate', 'conservation', 'conserve', 'conserved', 'conserves', 'conserving', 'contaminate', 'eco-activities', 'eco-friendly', 'ecology', 'emission', 'emit', 'energy', 'energy-efficient', 'environment', 'erode', 'erosion', 'externality', 'fertilis', 'fertiliz', 'greenlife', 'landscaping', 'natural', 'pollute', 'preserve', 'purify', 'recharge', 're-charge', 'recycle', 'salvage', 'solar', 'sustainability', 'sustainable', 'toxic', 'unpollute', 'unspoil', 'waste', 'water']
    list2 = ['clean', 'environmental', 'epa', 'sustainability', 'climate', 'warming', 'biofuel', 'biofuels', 'green', 'renewable', 'solar', 'stewardship', 'wind', 'atmosphere', 'emission', 'emissions', 'emit', 'ghg', 'ghgs', 'greenhouse', 'agriculture', 'deforestation', 'pesticide', 'pesticides', 'wetlands', 'zoning', 'biodiversity', 'species', 'wilderness', 'wildlife', 'freshwater', 'groundwater', 'water', 'cleaner', 'cleanup', 'coal', 'contamination', 'fossil', 'resource', 'air', 'carbon', 'nitrogen', 'pollution', 'superfund', 'biphenyls', 'hazardous', 'householding', 'pollutants', 'printing', 'recycle', 'recycling', 'toxic', 'waste', 'wastes', 'weee', 'climate change', 'conservation', 'environmentally', 'footprint', 'global warming', 'pollutant', 'recycled', 'sustainable', 'sustainably']
    environment_dictionary = list(set(list1 + list2))
    list11 = ['accountable', 'benefice', 'beneficiar', 'beneficial', 'communit', 'compassion', 'concerned', 'cooperate', 'development', 'educate', 'benefit', 'benevolen', 'brotherhood', 'care', 'caring', 'charit', 'civic', 'class', 'classes', 'concern', 'cultivating', 'empower', 'equal', 'equality', 'familial', 'families', 'family', 'freedom', 'graduation', 'happiness', 'happy', 'harmony', 'harvesting', 'harvests', 'health', 'help', 'humanity', 'humankind', 'immuniz', 'independent', 'joy', 'justice', 'kind', 'learn', 'liberate', 'liberty', 'life', 'mankind', 'partnership', 'peace', 'prosper', 'reading', 'responsibilities', 'rights', 'social', 'societ', 'SROI', 'success', 'support', 'teach', 'tender', 'trustworth', 'virtu', 'welfare', 'wellbeing', 'well-being', 'wisdom']
    list22 = ['citizen','citizens', 'csr','disabilities', 'disability', 'disabled', 'human', 'nations', 'social', 'un', 'veteran', 'veterans', 'vulnerable', 'dignity', 'discriminate', 'discriminated', 'discriminating', 'discrimination', 'equality', 'freedom', 'humanity', 'nondiscrimination', 'sexual', 'communities', 'community', 'expression', 'marriage', 'privacy', 'peace', 'bargaining', 'eeo', 'fairness', 'fla', 'harassment', 'injury', 'labor', 'overtime', 'ruggie', 'sick', 'wage', 'wages', 'workplace', 'bisexual', 'diversity', 'ethnic', 'ethnically', 'ethnicities', 'ethnicity', 'female', 'females', 'gay', 'gays', 'gender', 'genders', 'homosexual', 'immigration', 'lesbian', 'lesbians', 'lgbt', 'minorities', 'minority', 'ms', 'race', 'racial', 'religion', 'religious', 'sex', 'transgender', 'woman', 'women', 'occupational', 'safe', 'safely', 'safety', 'ilo', 'labour', 'eicc', 'children', 'epidemic', 'health', 'healthy', 'ill', 'illness', 'pandemic', 'childbirth', 'drug', 'medicaid', 'medicare', 'medicine', 'medicines', 'hiv', 'alcohol', 'drinking', 'bugs', 'conformance', 'defects', 'fda', 'inspection', 'inspections', 'minerals', 'standardization', 'warranty', 'endowment', 'endowments', 'people', 'philanthropic', 'philanthropy', 'socially', 'societal', 'society', 'welfare', 'charitable', 'charities', 'charity', 'donate', 'donated', 'donates', 'donating', 'donation', 'donations', 'donors', 'foundation', 'foundations', 'gift', 'gifts', 'nonprofit', 'poverty', 'courses', 'educate', 'educated', 'educates', 'educating', 'education', 'educational', 'learning', 'mentoring', 'scholarships', 'teach', 'teacher', 'teachers', 'teaching', 'training', 'employ', 'employment', 'headcount', 'hire', 'hired', 'hires', 'hiring', 'staffing', 'unemployment']
    social_dictionary = list(set(list11 + list22))
    from nltk.corpus import stopwords

    def remove_stopwords(words):
        # 불용어 및 특수문자 제거
        stop_words = set(stopwords.words('english'))
        filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
        return filtered_words
    # environment_dictionay와 social_dictionary에 불용어 제거만 수행
    environment_dictionary = remove_stopwords(environment_dictionary)
    social_dictionary = remove_stopwords(social_dictionary)
    # NLTK의 word_tokenize를 통해 단어 토큰화 수행 함수
    def tokenize(text):
        tokens = nltk.word_tokenize(text)
        # 불용어 및 특수문자 제거
        stop_words = set(stopwords.words('english'))
        tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
        return tokens


    # LDA에 포함할 리스트
    topiclist = []
    # dataset = [['Cozy Kornerz: Doing the Dezigning ☁️', "Cozy Kornerz is a fun new redecorating tool that'll help bring jaw dropping transformations to your space with ease!", 'cozy-kornerz-doing-the-dezigning', 'Anna-Marie Haynes'], ['Full Body Exercise Matt Double Sided', 'I want to design an exercise Matt with practice instructions that will increase your performance through out the day.', 'full-body-exercise-matt-double-sided', 'Milad Nazar'], ['creator name', 'blurb', 'name', 'slug']]
    for data in dataset:
        data = [tokenize(word) for word in data]
        # 빈 요소 제거
        data = [v for v in data if v]
        from gensim import corpora
        dictionary = corpora.Dictionary(data)
        corpus = [dictionary.doc2bow(text) for text in data]
        import gensim
        NUM_TOPICS = 1
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
        topics = ldamodel.print_topics(num_words=9999999)
        for topic in topics:
            topiclist.append(topic)
    # print(topiclist)
    # print(len(topiclist))
    # print(dataset)
    # ESG factor 계산
    from scipy.special import softmax
    # weight*"word" 형태를 weight과 word로 분리
    def split_weight_and_word(s):
        parts = s.split('"')
        weight = float(parts[0].strip('*').strip())
        word = parts[1]
        return weight, word

    def calculate_scores(l1, l2):
        scores = []

        for tuple_item in l1:
            # 튜플에서 weight과 string 추출
            weight, string = tuple_item

            # string을 토큰화
            sum = 0
            worddlist = []
            weightlist = []
            words = [word.strip('\"*') for word in string.split('+')]
            for weightandword in words:
                weight, word = split_weight_and_word(weightandword)
                weightlist.append(weight)
                worddlist.append(word)
            # softmax 함수 적용
            softmax_weightlist = softmax(weightlist)
            for i in range(len(worddlist)):
                if worddlist[i] in l2:
                    sum+=softmax_weightlist[i]

            # 총 score 계산
            scores.append(sum)


        return scores

    # 점수 계산 및 출력
    import math
    ESG_scores_list = []
    # ENV_list = calculate_scores(textstopic, environment_dictionary)
    # S_list = calculate_scores(textstopic, social_dictionary)
    ENV_list = calculate_scores(topiclist, environment_dictionary)
    S_list = calculate_scores(topiclist, social_dictionary)
    for i in range(len(ENV_list)):
        ENV_scores = ENV_list[i]
        S_scores = S_list[i]
        data_array[i]['ENV_scores'] = ENV_scores
        data_array[i]['S_scores'] = S_scores
        if ENV_scores == 0 and S_scores == 0:
            ESG_scores = 0
        else:
            ESG_scores = math.sqrt((ENV_scores**2 + S_scores**2)/2)
            # ESG_scores = (ENV_scores + S_scores)/2
            # ESG_scores = (ENV_scores + S_scores)/2
            data_array[i]['ESG_scores']=ESG_scores
        ESG_scores_list.append(ESG_scores)
    # print(len(ESG_scores_list))
    # ESG Score 평균 및 표준편차 계산
    import numpy
    print("mean:",numpy.mean(ESG_scores_list))
    print("std:", numpy.std(ESG_scores_list))
    with open(output_file_path, 'w') as new_file:
        json.dump(data_array, new_file, indent=4)
input_folder_path = "/Users/choeseoyeong/Desktop/raw_data"
output_folder_path = "/Users/choeseoyeong/Desktop/new_data"
process_json_files(input_folder_path, output_folder_path)

  Using cached pandas-1.3.1.tar.gz (4.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "/Users/choeseoyeong/anaconda3/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/Users/choeseoyeong/anaconda3/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Users/choeseoyeong/anaconda3/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/choeseoyeong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/choeseoyeong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/choeseoyeong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/choeseoyeong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


mean: 0.020962619096248056
std: 0.034120015174726695
mean: 0.025444851441606575
std: 0.03536433413810733
mean: 0.020917579448668646
std: 0.03328471680449449
mean: 0.02328286885613686
std: 0.03491067578404711
mean: 0.027071432095394313
std: 0.036089852429956215
mean: 0.01950747905125426
std: 0.03183680831186101
mean: 0.025064974905925644
std: 0.03549532835412434
mean: 0.02206040505873327
std: 0.03463747154911742
mean: 0.023232691580728815
std: 0.036029786920107026
mean: 0.02639829604939488
std: 0.03594011316974839
